In [118]:
from imblearn.ensemble import BalancedRandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import RFE
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from imblearn.keras import balanced_batch_generator
from imblearn.over_sampling import RandomOverSampler
import seaborn as sns
import tensorflow as tf
%matplotlib inline

In [98]:
features = pd.read_pickle("train_features.pkl")
labels = pd.read_csv("train_labels.csv",index_col="challenge_oid")
features.index.name = "challenge_oid"
features = features.join(labels)

In [99]:
labels = features["classALeRCE"]
del features["classALeRCE"] 

In [100]:
maps = {"SNIa":0, "SNII":1, "SNIbc":2, "RRL":3, 
             "EB":4, "DSCT":5, "Ceph":6, "Periodic-Other":7, 
             "AGN":8, "QSO":9, "YSO":10, "CV/Nova":11, "LPV":12, "Blazar":13}

colums = ["SNIa", "SNII", "SNIbc", "RRL", 
             "EB", "DSCT", "Ceph", "Periodic-Other", 
             "AGN", "QSO", "YSO", "CV/Nova", "LPV", "Blazar"]

In [101]:
new_classes = []
for old_class in labels.values:
    new_classes.append(maps[old_class])

In [150]:
inputs = tf.keras.layers.Input(len(features.columns)) 

layer1 = tf.keras.layers.Dense(128,activation="relu")(inputs)
layer2 = tf.keras.layers.Dense(64,activation="relu")(layer1)
layer3 = tf.keras.layers.Dense(32,activation="relu")(layer2)

layer_transient_1 = tf.keras.layers.Dense(16,activation="relu")(layer3)
layer_periodic_1 = tf.keras.layers.Dense(16,activation="relu")(layer3)
layer_stochastic_1 = tf.keras.layers.Dense(16,activation="relu")(layer3)


layer_transient_2 = tf.keras.layers.Dense(3,activation="sigmoid")(layer_transient_1)
layer_periodic_2 = tf.keras.layers.Dense(5,activation="sigmoid")(layer_periodic_1)
layer_stochastic_2 = tf.keras.layers.Dense(6,activation="sigmoid")(layer_stochastic_1)

outputs = tf.keras.layers.Concatenate()([layer_transient_2,layer_periodic_2, layer_stochastic_2])
outputs = tf.keras.layers.Softmax()(outputs)

model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

In [151]:
minmax = MinMaxScaler()
minmax.fit(features)
scaled_features = minmax.transform(features)
scaled_features = scaled_features+1
scaled_features = np.nan_to_num(scaled_features)-1 

In [152]:
y = np.array(new_classes)
y = tf.keras.utils.to_categorical(y)

In [153]:
training_generator, steps_per_epoch = balanced_batch_generator(
    scaled_features, y, sampler=RandomOverSampler(), batch_size=50, random_state=42)

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy",metrics=["accuracy"])
model.fit_generator(generator=training_generator,
                                       steps_per_epoch=100,
                                       epochs=50)

Epoch 1/50
100/100 [==============================] - 3s 25ms/step - loss: 2.5439 - accuracy: 0.1444
Epoch 2/50
100/100 [==============================] - 2s 25ms/step - loss: 2.3410 - accuracy: 0.2630
Epoch 3/50
100/100 [==============================] - 2s 24ms/step - loss: 2.2536 - accuracy: 0.3136
Epoch 4/50
100/100 [==============================] - 2s 25ms/step - loss: 2.2119 - accuracy: 0.3302
Epoch 5/50
100/100 [==============================] - 3s 25ms/step - loss: 2.1865 - accuracy: 0.3386
Epoch 6/50
100/100 [==============================] - 2s 24ms/step - loss: 2.1656 - accuracy: 0.3704
Epoch 7/50
100/100 [==============================] - 3s 25ms/step - loss: 2.1565 - accuracy: 0.3854
Epoch 8/50
100/100 [==============================] - 3s 26ms/step - loss: 2.1508 - accuracy: 0.3940
Epoch 9/50
100/100 [==============================] - 2s 24ms/step - loss: 2.1318 - accuracy: 0.4110
Epoch 10/50
100/100 [==============================] - 2s 25ms/step - loss: 2.1328 - accura

In [ ]:
test_features = pd.read_pickle("test_features.pkl")

In [ ]:
scaled_test_features = minmax.transform(test_features)
scaled_test_features = scaled_test_features+1
scaled_test_features = np.nan_to_num(scaled_test_features)-1 

In [ ]:
probs = model.predict(scaled_test_features)

In [ ]:
probs = pd.DataFrame(probs, columns=colums)

In [ ]:
probs["Outlier"] = 0

In [ ]:
probs.index = test_features.index

In [ ]:
probs